In [1]:
from __future__ import division

from afinn import Afinn
import numpy as np
import pandas as pd

In [13]:
# http://www.crowdflower.com/data-for-everyone
url = 'http://cdn2.hubspot.net/hubfs/346378/DFE_CSVs/Twitter-sentiment-self-drive-DFE.csv?t=1436301804871'
df = pd.read_csv(url)
df

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,sentiment,sentiment:confidence,our_id,sentiment_gold,sentiment_gold_reason,text
0,724227031,True,golden,236,NaN,5,0.7579,10001,5\n4,Author is excited about the development of the...,Two places I'd invest all my money if I could:...
1,724227032,True,golden,231,NaN,5,0.8775,10002,5\n4,Author is excited that driverless cars will be...,Awesome! Google driverless cars will help the ...
2,724227033,True,golden,233,NaN,2,0.6805,10003,2\n1,The author is skeptical of the safety and reli...,If Google maps can't keep up with road constru...
3,724227034,True,golden,240,NaN,2,0.8820,10004,2\n1,The author is skeptical of the project's value.,Autonomous cars seem way overhyped given the t...
4,724227035,True,golden,240,NaN,3,1.0000,10005,3,Author is making an observation without expres...,Just saw Google self-driving car on I-34. It w...
5,724227036,True,golden,241,NaN,3,1.0000,10006,3,Author is asking a question without expressing...,Will driverless cars eventually replace taxi d...
6,724227037,True,golden,226,NaN,not_relevant,0.5367,10007,not_relevant,Trains (metros) are not relevant to the focus ...,Chicago metro expected to be fully autonomous ...
7,724227038,True,golden,241,NaN,not_relevant,0.6548,10008,not_relevant,Author is not referring to self-driving cars. ...,I love the infotainment system in my new car. ...
8,724227039,True,golden,238,NaN,5,0.7187,10009,5\n4,Shows excitement that autonomous vehicles will...,Autonomous vehicles could reduce traffic fatal...
9,724227040,True,golden,230,NaN,1,0.6412,10010,2\n1,Shows fear that driverless cars will not be sa...,Driverless cars are not worth the risk. Don't...


In [12]:
# Sentiment analysis with AFINN
afinn = Afinn(emoticons=True)
afinn_scores = [afinn.score(text) for text in df.text]
df['afinn'] = afinn_scores
df[['sentiment', 'afinn', 'text']].head(10)

,sentiment,afinn,text
0,5,0,Two places I'd invest all my money if I could:...
1,5,5,Awesome! Google driverless cars will help the ...
2,2,1,If Google maps can't keep up with road constru...
3,2,0,Autonomous cars seem way overhyped given the t...
4,3,0,Just saw Google self-driving car on I-34. It w...
5,3,0,Will driverless cars eventually replace taxi d...
6,not_relevant,0,Chicago metro expected to be fully autonomous ...
7,not_relevant,3,I love the infotainment system in my new car. ...
8,5,-3,Autonomous vehicles could reduce traffic fatal...
9,1,1,Driverless cars are not worth the risk. Don't...


In [4]:
pd.crosstab(df.sentiment, df.afinn)

afinn,-11.0,-10.0,-9.0,-8.0,-7.0,-6.0,-5.0,-4.0,-3.0,-2.0,...,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,12.0
sentiment,,,,,,,,,,,,,,,,,,,,,
1,1,1,0,1,4,5,2,13,16,12,...,6,5,1,0,2,0,0,0,0,0
2,0,1,2,4,7,7,11,32,53,115,...,66,38,21,4,6,1,0,0,1,0
3,0,0,0,1,3,5,15,39,84,211,...,316,157,85,24,10,8,5,0,2,0
4,1,0,0,1,3,7,10,24,40,87,...,177,151,90,34,22,7,9,1,0,0
5,0,0,1,0,1,0,0,5,14,14,...,55,70,61,13,14,4,5,2,1,1
not_relevant,0,0,1,0,1,2,2,4,2,9,...,22,15,12,4,4,2,0,0,1,0


In [5]:
# Ignore tweets that are marked as 'not relevant'
df_relevant = df[df.sentiment != 'not_relevant']
df_relevant = df_relevant.convert_objects(convert_numeric=True)

In [6]:
# Compute 3-class confusion matrix
confusion = pd.crosstab(np.sign(df_relevant.sentiment - 3), np.sign(df_relevant.afinn))
confusion

afinn,-1.0,0.0,1.0
sentiment,,,
-1,357,213,225
0,601,2698,946
1,304,617,982


In [7]:
accuracy_3_class = np.sum(np.diag(confusion)) / np.sum(confusion.values)
accuracy_3_class

0.58144894137980696

In [9]:
#Compute 2-class confusion matrix
confusion_2_class = confusion.iloc[[0, 2], [0, 2]]
confusion_2_class

afinn,-1.0,1.0
sentiment,,
-1,357,225
1,304,982


In [10]:
accuracy_2_class = np.sum(np.diag(confusion_2_class)) / np.sum(confusion_2_class.values)
accuracy_2_class

0.71680942184154173

In [11]:
accuracy_2_class_baseline = confusion_2_class.sum().max() / np.sum(confusion_2_class.values)
accuracy_2_class_baseline 

0.64614561027837258